# **INITIALIZATION**

In [14]:
#%pip install -r ./requirements/requirements.txt

In [15]:
from os import chdir

# à modifier selon utilisation du serveur de google colab (is_local=False)
# ou de l'ordinateur sur lequel vous travaillez (is_local=True)
is_local = True

# pour le moment, seuls le français, l'anglais, l'espagnol, l'allemand,
# le catalan, le chinois, le danois, le japonais, le slovaque et l'ukrainien sont gérés.
# fr, en, es, de, ca, zh, da, ja, sl, uk
language = 'fr'

if not is_local:
    from google.colab import drive
    drive.mount('/content/drive/')
    folder_path = '/content/drive/MyDrive/COLAB_NLP/'
    chdir(folder_path)
    %pip install -r ./requirements/requirements.txt
else:
    folder_path = './'

In [ ]:
import spacy
import torch

if torch.cuda.is_available():
    print("GPU found")
    spacy.require_gpu()
else:
    print("No GPU available")

In [16]:
file_to_run = f"{folder_path}all_main_functions.ipynb"
%run "$file_to_run"

In [17]:
#%%capture

import os
from urllib import request
import subprocess
import zipfile

download_things()

go seaborn
go gensim
go cleantext
go adjustText
go umap
go nltk
go sklearn
go scipy
go matplotlib
go spacy
go spacy-transformers
go pyate
go multiprocess
go bs4
go unidecode
go statsmodels
Language fr downloaded


In [18]:
imports_file_path = f"{folder_path}imports.ipynb"
%run "$imports_file_path"

In [19]:
matplotlib.rcParams['font.family'] = 'Liberation Mono'
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['savefig.dpi'] = 300

# **PARAMETERS DEFINITION**

In [20]:
# base_name correspond au nom du corpus. c'est la variable qu'il faut modifier de sorte
# à caractériser précisément chaque corpus, avec les bonnes pratiques qui conviennent
# (par exemple : "ukraine_russie__presse_francilienne__fev2022_feb2023").
# le contenu de la variable base_name doit être retrouvé à l'intérieur des noms de fichiers,
# à l'intérieur du dossier "DATA", sur lesquels l'analyse sera produite.
# l'algorithme gère les corpus divisés en plusieurs fichiers : il suffit que
# le contenu de la variable base_name soit présent dans tous les fichiers concernés par l'analyse.
# par exemple, un fichier "ukraine_russie__presse_francilienne__fev2022_mars2024__feb2022_sep2022.HTML"
# et un fichier "ukraine_russie__presse_francilienne__fev2022_feb2023__oct2022_feb2023.HTML"
base_name = 'transcripts'

results_path = folder_path + "RESULTS_" + base_name + "/"

if not os.path.exists(results_path):
    os.makedirs(results_path)

name_document = f'{base_name}.HTML'
raw_documents_output_name = results_path + f'{base_name}_raw_documents.txt'
documents_info_name = results_path + f'{base_name}_info'
documents_lemmas_info_name = results_path + f'{base_name}_lemmas_info'
lemmatized_documents_output_name = results_path + f'{base_name}_lemmatized_documents.txt'

topic_model_unigrams_output_name = results_path + f'{base_name}_unigrams_topic_model'
topic_model_sentences_output_name = results_path + f'{base_name}_sentences_topic_model'
topic_model_terms_output_name = results_path + f'{base_name}_terms_topic_model'
topic_model_entities_output_name = results_path + f'{base_name}_entities_topic_model'

# ce sont des valeurs qui ne servent que pour les corpus issus d'europresse.
# les articles de moins de 500 caractères et de plus de 100000 caractères sont
# considérés comme suspicieux, et supprimés.
# important à modifier. selon le type de corpus, ces valeurs devraient considérablement varier.
# avec des corpus issus d'europresse, l'algorithme supprimerait du corpus tous les articles
# avec moins de minimum_caracters_nb_by_document caractères et plus de maximum_caracters_nb_by_document caractères.
# il y a à cela deux raisons. d'une part, de la suspicion : qu'est-ce qu'un article avec moins, par exemple, de 500
# caractères, et avec plus de 100000 caractères ? ce n'est en tout cas clairement pas standard.
# d'autre part, il faut du thème. avec moins de 100 ou 200 caractères, globalement, c'est-à-dire avec moins de vingt mots environ,
# il est difficile de qualifier un document avec son vocabulaire seul (c'est le même principe que
# lorsqu'on disqualifie un chi2 du fait de croisements dont l'effectif est insuffisant).
minimum_caracters_nb_by_document = 250
maximum_caracters_nb_by_document = 100000

# sub_linear_tfidf est un paramètre aux implications importantes.
# avant la factorisation de matrices, grâce à laquelle la détermination des topics/thèmes,
# l'algorithme attribue à chaque unigramme lemmatisé un score tf-idf
# (term frequency - inverse document frequency).
# ce score calcule à quel point un unigramme est présent dans le document,
# mais rare dans les autres. c'est un score de spécificité : il approxime à quel point
# l'unigramme singularise le document dans lequel il se trouve.
# sub_linear_tfidf applique une fonction logarithmique à la fréquence à laquelle
# l'unigramme a été trouvé dans le document. le score tf-idf met alors en valeur les mots qui sont spécifiques,
# principalement parce qu'ils sont rares dans les autres documents.
# le score est alors moins sensible à la fréquence à laquelle l'unigramme est retrouvé dans le document.
sub_linear_tfidf = False
unigrams_nb_by_topic = 100

is_europresse = False

# la valeur indique si les doublons doivent être ou non conservés (à False, ils sont conservés ;
# à True, ils sont retirés). la modalité "True" est probablement importante pour les corpus
# issus d'Europresse. nous évitons de cette manière les éventuels problèmes de doublonnage
# liés à la manière même dont Europresse fonctionne. nous évitons également les problèmes
# possibles de doublonnage au cas où nous ayons par inadvertance des fichiers dont les
# articles se recoupent en partie. nous retirons finalement tous les articles produits
# par copié/collé d'autres articles. l'algorithme résiste en effet aux variations marginales :
# les articles qui n'apportent rien de significativement nouveau sont ainsi supprimés du corpus.
# il n'est toutefois pas absurde de vouloir les conserver (le paramètre devra alors être laissé à False).
# les copié/collé participent en effet du paysage médiatique auquel nous nous intéressons.
go_remove_duplicates = False

fontsize = 16

# **DOCUMENTS PREPARATION**

In [21]:
documents = []
all_soups = []
columns_dicts = {}
meta_load_documents()

DOCUMENTS PROCESSED: |          | 0/? [00:00<?, ?it/s]


In [22]:
# AFTER OUTLIERS REMOVAL
print(len(documents))

441


In [23]:
if go_remove_duplicates:
    remove_duplicates()

In [24]:
# AFTER DUPLICATES REMOVAL
print(len(documents))

441


In [25]:
write_raw_documents()

#**DOCUMENTS PROCESSING**

In [26]:
documents_lemmatized = []
all_tab_pos = []
sentences_norms = []
sentences_lemmas = []
treat_documents_pipe()

Model loaded in 4.7481889724731445s


100%|██████████| 441/441 [07:45<00:00,  1.06s/it]  


In [27]:
write_lemmatized_documents()

In [28]:
write_all_pickles()

In [29]:
print(len(documents_lemmatized))

441


# **SAVE POINT**

In [30]:
# possibilité de reprendre l'analyse ici si le traitement a déjà opéré.
# il faut toutefois d'abord passer tous les blocs de l'initialisation, et
# de la définition du paramètre. la préparation et le processing des documents
# peuvent alors être biaisés.
documents_lemmatized, all_tab_pos, sentences_norms, sentences_lemmas = read_all_pickles()

#**UNIGRAMS EXTRACTION**

In [31]:
unigrams = {}
unigrams_nouns = {}
unigrams_common_nouns = {}
unigrams_proper_nouns = {}
determine_unigrams()

#**TF-IDF**

In [32]:
tfidf_vectorizer, tfidf, tfidf_feature_names, tokenized_documents = go_tfidf_vectorization(unigrams)

#**UMAP**

In [33]:
# unigrams_nb préside au nombre d'unigrammes retenus pour l'analyse.
# n_neighbors donne une idée de la contrainte "structurelle" :
# à quel point umap ira chercher beaucoup
create_umap_best_ngrams(unigrams_nb=200, n_neighbors=5)

#**NMF**

In [34]:
all_nmf_H = {}
all_nmf_W = {}
relevant_i_by_topics = {}
every_topics_and_scores_by_document = {}
determine_nmf(initial_topic_num=5, terminal_topic_num=20)

TOPICS CONFIGURATIONS PROCESSED: 100%|██████████| 4/4 [00:00<00:00, 23.46it/s]


In [35]:
write_documents_infos()
write_topics_unigrams()

100%|██████████| 4/4 [00:00<00:00, 52.74it/s]


#**SENTENCES EXTRACTION**

In [36]:
all_sentences_array = {}
all_sentences_array_original = {}
sentences_extraction()
write_topics_sentences()

TOPICS CONFIGURATIONS PROCESSED: 100%|██████████| 4/4 [00:00<00:00, 667.09it/s]


#**TERMS AND ENTITIES EXTRACTION**

In [37]:
terms_and_entities_extraction()

TOPICS CONFIGURATIONS PROCESSED: 100%|██████████| 4/4 [00:37<00:00,  9.42s/it]


#**TOPICS DYNAMICS**

In [38]:
original_labels = {5:  ["0", "1", "2", "3", "4"],
                   10: ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"],
                   15: ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14"],
                   20: ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19"]}

In [39]:
# le paramètre sigma détermine le niveau de lissage. la valeur 'auto' essaie de déterminer
# le lissage "optimal" à partir de la moyenne des écart-types propres à la distribution de
# chaque topic. l'automatisation du lissage ne fonctionne toutefois pas toujours, et il est bon
# de tâtonner entres différentes valeurs numériques de sigma (si sigma=1, il n'y a pas de lissage).
# il n'y a de toute façon pas de lissage "optimal" en soi. il y a bien une sorte de lissage intermédiaire
# grâce auquel produire immédiatement des effets d'intelligibilité : c'est le niveau de lissage à partir duquel
# se distinguent clairement la dynamique des topics (ceux qui sont surtout représentés au début du corpus, et ceux
# qui sont surtout représentés à la fin). il faut toutefois faire attention : ce niveau de lissage tend à masquer de fortes
# disparités à l'intérieur de la distribution. une fois la distribution lissée, il n'est plus possible de savoir
# si une grande zone forte l'est du fait que le topic soit globalement présent tout du long de la période,
# ou si elle l'est du fait d'un ou quelques événements extrêmement intenses.
# les deux autres paramètres président à la normalisation de la distribution.
# normalize_by_date neutralise dans la distribution les effets dû aux différences du nombre d'articles
# publiés à une date donnée. le traitement médiatique de la guerre en ukraine est un exemple impeccable :
# sans normalize_by_date=True, tous les topics seraient sur-représentés les premiers jours de la guerre.
# c'est un simple effet de la masse de publications ces jours-là. normalize_by_date=True montre ainsi une
# une distribution indépendante de la masse de la publication (ce qui ne veut pas dire que l'absence
# de normalisation par la masse journalière de publication n'est pas digne d'intérêt).
# relative_normalizaton détermine si chaque distribution de chaque topic a sa norme propre,
# ou si chaque distribution partage avec les autres une norme d'ensemble. tous les topics ne sont en effet pas
# représentés de la même manière dans le corpus. certains sont beaucoup plus présents que d'autres.
# de ce fait et en l'absence de normalisation "relative", les topics les plus représentés et les moins représentés
# s'écraseraient mutuellement. dit autrement : si la différence d'intensité entre les topics les plus représentés
# et les topics les moins représentés est trop forte, les topics les plus représentés ne seraient plus que de longues bandes
# bleues, et les topics les moins représentés, de longues bandes jaunes. les différences internes à ces distributions
# seraient écrasées. la normalisation "relative" corrige ce problème de visibilité s'il en est : chaque distribution de
# chaque topic est ainsi normalisée par sa norme propre de sorte à ce que pour chaque distribution, la valeur maximale
# soit toujours 1, et la valeur minimale, 0. s'observe de cette manière la dynamique "propre" du topic.
# cela ne veut toujours pas dire que l'absence de normalisation "relative" est inintéressante : ne pas normaliser de cette façon
# aide à ne pas sur-interpréter les distributions normalisées. si la distribution a été normalisée,
#ce n'est pas parce qu'un topic semble fort à un moment donné qu'il l'est dans l'absolu.
create_chrono_topics(normalize_by_date=False, relative_normalizaton=True, sigma=10)

TOPICS CONFIGURATIONS PROCESSED:   0%|          | 0/4 [00:00<?, ?it/s]

TOPICS CONFIGURATIONS PROCESSED: 100%|██████████| 4/4 [00:08<00:00,  2.17s/it]


#**TOPICS RANKING BARPLOTS**

In [40]:
len(all_soups)

0

In [41]:
%%capture
# plus une barre est bleue, plus l'écart-type de la distribution à laquelle
# elle correspond est faible. autrement dit, plus une barre est bleue, plus
# le thème est également distribuée entre les documents. et inversement :
# plus une barre est rouge, plus sa distribution est asymétrique. le thème est
# très présent sur un ou quelques documents, et très peu voire pas du tout sur les autres.
create_topic_barplots()

# **DATA PREPARATION**

In [42]:
# Structure modifiée pour une meilleure lisibilité et maintenance
distri_topics_by_journal_by_num_topic = {}

for num_topic in every_topics_and_scores_by_document:
    # Initialiser un dictionnaire pour stocker la distribution des sujets par journal pour ce topic
    distri_topics_by_journal = {}

    for num_article in every_topics_and_scores_by_document[num_topic]:
        # Extrait le nom du journal de l'article et le normalise
        header = all_soups[num_article].header
        journal_text = extract_information(header, '.rdp__DocPublicationName')
        journal_text = normalize_journal(journal_text)

        # Itère sur chaque tuple sujet-score dans l'article
        for topic_score_tuple in every_topics_and_scores_by_document[num_topic][num_article]:
            topic, score = topic_score_tuple  # Décomposer le tuple

            # Met à jour le dictionnaire pour ce sujet
            if topic not in distri_topics_by_journal:
                distri_topics_by_journal[topic] = {}

            if journal_text not in distri_topics_by_journal[topic]:
                distri_topics_by_journal[topic][journal_text] = []

            distri_topics_by_journal[topic][journal_text].append(score)

    # Met à jour le dictionnaire principal avec la distribution des sujets pour ce topic
    distri_topics_by_journal_by_num_topic[num_topic] = distri_topics_by_journal

IndexError: list index out of range

# **KRUSKAL-WALLIS TEST**

In [ ]:
threshold = 10

In [ ]:
kruskal_results = {}

In [ ]:
for num_topic in distri_topics_by_journal_by_num_topic:
    for topic in distri_topics_by_journal_by_num_topic[num_topic]:
        topic_data = distri_topics_by_journal_by_num_topic[num_topic][topic]

        # Collecte des données de score pour chaque journal sous ce sujet et topic
        data = []
        journals = []  # Pour les étiquettes
        for journal, scores in topic_data.items():
            data.extend(scores)
            journals.extend([journal] * len(scores))

        # Création d'un DataFrame pour Seaborn
        df = pd.DataFrame({'Journal': journals, 'Score': data})

        # Compter les occurrences de chaque journal
        journal_counts = df['Journal'].value_counts()

        # Filtrer les journaux avec moins de "threshold" occurrences
        journals_to_keep = journal_counts[journal_counts >= threshold].index
        df = df[df['Journal'].isin(journals_to_keep)]

        # Vérifie s'il y a au moins deux groupes pour effectuer le test
        if len(set(df['Journal'])) < 2:
            print("Pas assez de groupes pour effectuer le test pour ce sujet et topic")
            continue

        # Effectuer le test de Kruskal-Wallis
        stat, p = stats.kruskal(*[group['Score'] for name, group in df.groupby('Journal')])

        kruskal_results[(num_topic, topic)] = {'statistic': stat, 'p_value': p}

        unique_journals = df['Journal'].unique()
        max_ylim = [float('inf'), float('-inf')]  # Initialiser avec des valeurs extrêmes

        # Première itération pour déterminer la largeur nécessaire et les limites de l'axe des y
        for journal in unique_journals:
            journal_data = df[df['Journal'] == journal]['Score']
            ax_temp = sns.violinplot(y=journal_data) #, ax=axes[i])
            ylim = ax_temp.get_ylim()
            max_ylim = [min(max_ylim[0], ylim[0]), max(max_ylim[1], ylim[1])]  # Mettre à jour les limites de l'axe des y
            plt.close()  # Fermer le plot temporaire

        max_ylim[1] = max_ylim[1] - (max_ylim[1] / 5)


       # sns.set_theme(context='notebook', style='white')

        # Créer la figure et le GridSpec
        fig = plt.figure(figsize=(15, 10), constrained_layout=True) #(20*(math.sqrt(len(unique_journals)/3)), 7))

        # Ajout du titre
        fig.suptitle(f"diagrammes en violon pour le topic '{original_labels[num_topic][topic]}' "
                    f"(configuration à {num_topic} topics) \n statistique de kruskal-wallis={stat:.3f}, "
                    f"p={p:.3f}")

        gs = GridSpec(1, len(unique_journals))

        # Calculer la moyenne des scores pour chaque journal
        mean_scores = df.groupby('Journal')['Score'].mean()

        # Normaliser les moyennes des scores
        normalized_mean_scores = (mean_scores - mean_scores.min()) / (mean_scores.max() - mean_scores.min())

        # Créer la palette de couleurs
        palette = sns.color_palette("flare", as_cmap=True)

        # Appliquer la palette de couleurs aux moyennes des scores normalisées
        mean_score_colors = normalized_mean_scores.apply(lambda x: palette(x))

        # Tracer les plots avec des largeurs ajustées et des limites de l'axe des y uniformes
        for i, journal in enumerate(unique_journals):
            ax = fig.add_subplot(gs[i])
            journal_data = df[df['Journal'] == journal]['Score']
            color = mean_score_colors[journal]
            sns.violinplot(y=journal_data,
                           ax=ax,
                           cut=0,
                           color=color,
                           linewidth=0,
                           width=1.0,
                           saturation=1.0,
                           gridsize=1000)
            ax.set_ylim(max_ylim)
            ax.set_xticklabels([])
            ax.set_xlabel('')
            if i == 0:
                ax.set_ylabel('')
            else:
                ax.set_ylabel('')
                ax.set_yticklabels([])

            # Désactiver les bordures du plot
            for spine in ax.spines.values():
                spine.set_visible(False)

            # Filtrer pour n'afficher que les ticks positifs sur l'axe des y
            yticks = ax.get_yticks()
            ax.set_yticks([ytick for ytick in yticks if ytick >= 0])

            # Ajouter les noms des journaux à la verticale
            ax.annotate(journal, xy=(0, 0), xytext=(0, 0),
                        textcoords='offset points', ha='center', va='top', rotation=90)

#        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
 #       plt.tight_layout(pad=1.0, h_pad=1.0, w_pad=1.0, rect=[0, 0, 1, 1])

        plt.savefig(f"{results_path}{base_name}_{num_topic}nt_{topic}t_{threshold}thres_journals_violin_plots.png",
                    dpi=300,
                    bbox_inches='tight',
                    pad_inches=0)

        plt.close()

In [ ]:
# Conversion en DataFrame
kruskal_df = pd.DataFrame([(key[0], key[1], val['statistic'], val['p_value']) for key, val in kruskal_results.items()],
                          columns=['Num_Topic', 'Topic', 'Statistique', 'P_value'])

# Tri et formatage
kruskal_df.sort_values(by='Statistique', ascending=False, inplace=True)
kruskal_df['Statistique'] = kruskal_df['Statistique'].round(3)
kruskal_df['P_value'] = kruskal_df['P_value'].apply(lambda x: f"{x:.3e}" if x < 0.001 else f"{x:.3f}")

# Pivoter le DataFrame pour la heatmap
heatmap_data = kruskal_df.pivot(columns=["Num_Topic", "Topic", "Statistique"])

# Création de la heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=False, cmap="coolwarm", linewidths=.5)

plt.title("heatmap des statistiques de kruskal-wallis")
plt.ylabel("configurations en nombres de topics")
plt.xlabel("numéros des topics")

plt.savefig(f"{results_path}{base_name}_{threshold}thres_kruskal_heatmap.png")

plt.close()

# **CHI2 TEST**

In [ ]:
chi2_results = []

# Traitement pour chaque num_topic
for num_topic in every_topics_and_scores_by_document.keys():
    data = []

    # Parcourir les articles de ce num_topic
    for num_article, topic_scores in every_topics_and_scores_by_document[num_topic].items():
        # Trouver le topic avec le score le plus élevé pour cet article
        highest_score_topic = max(topic_scores, key=lambda item: item[1])[0]

        # Extraire le nom du journal
        header = all_soups[num_article].header
        journal_text = extract_information(header, '.rdp__DocPublicationName')
        journal_text = normalize_journal(journal_text)

        # Ajouter au tableau de données
        data.append([journal_text, highest_score_topic])

    # Créer un DataFrame pour ce num_topic
    df = pd.DataFrame(data, columns=["Journal", "Topic"])

    # Compter les occurrences de chaque journal
    journal_counts = df['Journal'].value_counts()

    # Filtrer les journaux avec moins de occurrences que le nombre déterminé par le seuil
    journals_to_keep = journal_counts[journal_counts >= threshold].index
    df = df[df['Journal'].isin(journals_to_keep)]

    # Si pas assez de journaux, passer à l'itération suivante
    if len(journals_to_keep) < 2:
        print(f"num_topic: {num_topic} - Pas assez de journaux avec au moins 10 occurrences pour effectuer le test")
        continue

    # Créer un tableau de contingence pour ce num_topic
    contingency_table = pd.crosstab(df['Journal'], df['Topic'])

    #print(contingency_table)

    # Exécuter le test du chi2 pour ce num_topic
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # Ajouter les résultats dans la liste
    chi2_results.append((num_topic, chi2, p))

# Créer un DataFrame à partir de la liste des résultats
results_df = pd.DataFrame(chi2_results, columns=["topics_nb", "chi2", "p_value"])

# Enregistrer le DataFrame dans un fichier CSV
results_df.to_csv(f"{results_path}{base_name}_{threshold}thres_chi2.csv", index=False)

# **MULTINOMIAL REGRESSION**

In [ ]:
# Itération sur chaque topic et chaque article
for num_topic, articles in every_topics_and_scores_by_document.items():
    if num_topic == 5:
        data_for_regression = []

        for num_article, topics_scores in articles.items():
            # Extraction et normalisation du nom du journal
            header = all_soups[num_article].header
            journal_text = extract_information(header, '.rdp__DocPublicationName')
            journal_text = normalize_journal(journal_text)

            # Création de l'entrée pour cet article
            article_data = {'journal': journal_text}
            article_data.update(topics_scores)
            data_for_regression.append(article_data)

        # Conversion en DataFrame
        df = pd.DataFrame(data_for_regression)

        # Gestion des valeurs manquantes
        df.fillna(0, inplace=True)  # Remplace les valeurs manquantes par 0

In [ ]:
# Vérification de la Multicollinéarité
# Calcul du VIF (Facteur d'Inflation de la Variance)
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
    return vif_data

In [ ]:
multinomial_regression_num_topic = 5

In [ ]:
# Itération sur chaque topic et chaque article
for num_topic, articles in every_topics_and_scores_by_document.items():
    if num_topic == multinomial_regression_num_topic:
        data_for_regression = []

        for num_article, topics_scores in articles.items():
            # Extraction et normalisation du nom du journal
            header = all_soups[num_article].header
            journal_text = extract_information(header, '.rdp__DocPublicationName')
            journal_text = normalize_journal(journal_text)

            # Création de l'entrée pour cet article
            article_data = {'journal': journal_text}
            article_data.update(topics_scores)
            data_for_regression.append(article_data)

        # Conversion en DataFrame
        df = pd.DataFrame(data_for_regression)

        # Gestion des valeurs manquantes
        df.fillna(0, inplace=True)  # Remplace les valeurs manquantes par 0

        # Calculer la fréquence de chaque journal
        frequences = df['journal'].value_counts()

        # Identifier les journaux peu représentés
        journaux_peu_representes = frequences[frequences < threshold].index

        # Supprimer les lignes contenant ces journaux peu représentés
        df = df[~df['journal'].isin(journaux_peu_representes)]

        # Histogrammes pour les Variables Numériques
        df.hist(figsize=(12, 10))
        plt.tight_layout()
        plt.show()


        compte_classes = df['journal'].value_counts()

        # Calculer les proportions
        proportions_classes = compte_classes / len(df)

        # Afficher les compte et proportions
        print("Compte des classes:")
        print(compte_classes)
        print("\nProportions des classes:")
        print(proportions_classes)

        # Visualiser la distribution des classes
        plt.bar(compte_classes.index, compte_classes.values)
        plt.xlabel('Classe')
        plt.ylabel('Nombre d’occurrences')
        plt.title('Distribution des Classes')
        plt.xticks(rotation=90)
        plt.show()

        # Création de variables dummy pour les journaux
        journal_dummies = pd.get_dummies(df['journal'])
        df = pd.concat([df, journal_dummies], axis=1)

        print('REMAINING DOCUMENTS NB', len(df))


        # Définition des variables indépendantes et dépendantes
        X = df.drop(['journal'] + list(journal_dummies.columns), axis=1)  # Scores des topics

        vif_data = calculate_vif(X)
        print(vif_data)

        y = df[list(journal_dummies.columns)]  # Journaux en tant que variables dummy

        # Ajout d'une constante à X pour le terme d'intercept
        X = sm.add_constant(X)

        # Construction et ajustement du modèle de régression multinomiale
        model = sm.MNLogit(y, X)
        result = model.fit(method='bfgs', maxiter=5000, tol=1e-5)

        # Affichage des résultats
       # print(result.summary())

        # Supposons que 'result' est l'objet contenant les résultats de votre régression logistique multinomiale
        summary = result.summary()

        # Enregistrer le résumé dans un fichier texte
        with open(f"{results_path}{base_name}_{num_topic}t_regression_full_summary.txt", 'w') as fh:
            fh.write(summary.as_text())

        # Enregistrer le tableau de résultats dans un fichier CSV
        with open(f"{results_path}{base_name}_{num_topic}t_{threshold}thres_regression_results.csv", 'w') as fh:
            fh.write(summary.tables[1].as_csv())

# **SENTIMENT ANALYSIS**

In [ ]:
logging.basicConfig(level=logging.ERROR)

In [ ]:
logger = logging.getLogger('transformers')
logger.setLevel(logging.ERROR)

In [ ]:
%%capture

model_name = 'nlptown/bert-base-multilingual-uncased-sentiment' # Modèle spécifique pour l'analyse de sentiments
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sentiment_pipeline = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
def analyze_sentiment(text):
    try:
        # Tokeniser le texte pour compter les tokens
        tokens = tokenizer.tokenize(text)

        # Diviser le texte en segments, en respectant la limite de tokens
        max_tokens = 400 - tokenizer.num_special_tokens_to_add()  # Compte pour [CLS] et [SEP]
        token_chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]

        # Convertir les segments de tokens en chaînes de caractères
        text_chunks = [tokenizer.convert_tokens_to_string(chunk) for chunk in token_chunks]

        # Analyser le sentiment de chaque segment avec une barre de progression
        results = []
        for chunk in text_chunks:
            segment_result = sentiment_pipeline(chunk)
            results.append(segment_result)

        # Combiner les résultats
        # La logique de combinaison dépend de vos besoins spécifiques

        return results
    except Exception as e:
        print(f"Error in sentiment analysis: {e}")
        return None

# Imaginons que 'documents' est votre tableau de textes
sentiments = [analyze_sentiment(doc) for doc in tqdm(documents, desc="Processing Documents")]

In [ ]:
def extract_and_convert_date(date_str):
    # Utilisation d'une expression régulière pour extraire la date
    match = re.search(r'\d{1,2}/\d{1,2}/\d{4}', date_str)
    if match:
        return datetime.strptime(match.group(), '%d/%m/%Y')
    else:
        # Gérer les cas où aucune date n'est trouvée
        return None

In [ ]:
dates = []
if is_europresse:
    for soup in all_soups:
        header = soup
        date_text = extract_information(header, '.DocHeader')
        date_text_clean = extract_date_info(date_text)
        date_normalized = normalise_date(date_text_clean).replace(';', '').replace('&', '')
        date_normalized = extract_date_from_text(date_normalized)
        dates.append(date_normalized)
else:
    from dateutil.parser import parse

    def detecter_date(chaine, jour_en_premier=True):
        try:
            return parse(chaine, dayfirst=jour_en_premier)
        except ValueError:
            return None

    def formater_date(date):
        return date.strftime('%d/%m/%Y')

    def formater_liste_dates(liste_dates, jour_en_premier=True):
        return [formater_date(detecter_date(date_str, jour_en_premier)) for date_str in liste_dates if detecter_date(date_str, jour_en_premier)]

    dates = formater_liste_dates(columns_dicts['date'])

# Filtrer et convertir les dates
new_dates = []
for date_str in dates:
    date = extract_and_convert_date(date_str)
    new_dates.append(date)

dates = new_dates

In [ ]:
# Transformer les sentiments en scores basés sur les étoiles
transformed_sentiments = []
for doc_sentiments in sentiments:
    if doc_sentiments:  # Assurez-vous que la liste n'est pas vide
        doc_scores = []
        for sentiment_list in doc_sentiments:
            sentiment = sentiment_list[0]  # Accéder au premier élément de la liste
            label = sentiment['label']
            star_rating = int(label.split()[0])  # Extraire le nombre d'étoiles

            doc_scores.append(star_rating)

        average_score = sum(doc_scores) / len(doc_scores)
        transformed_sentiments.append(average_score)
    else:
        transformed_sentiments.append(None)  # ou une valeur par défaut

# Supposons que `dates` est votre liste de dates sous forme de chaînes de caractères
import pandas as pd
df = pd.DataFrame({'date': dates, 'sentiment': transformed_sentiments})

# Convertir les chaînes de caractères en type datetime
df['date'] = pd.to_datetime(df['date'])

# Trier le DataFrame par date
df = df.sort_values(by='date')

# Grouper par date et calculer la moyenne des sentiments par jour
df_grouped = df.groupby('date').mean()

# **SENTIMENT OVER TIME**

In [ ]:
# Supposons que 'Valeur' est le nom de la colonne de données que vous voulez lisser
y = df_grouped['sentiment'].values
x = df_grouped.index.values

# Appliquer le filtre de Savitzky-Golay
window_size = 41  # La taille de la fenêtre doit être impaire
polynomial_order = 3  # L'ordre du polynôme utilisé pour l'ajustement
yhat = savgol_filter(y, window_size, polynomial_order)

# Créer le graphique
#plt.plot(x, y, label='Original')
plt.plot(x, yhat, color='blue')
plt.title('dynamique des sentiments')

plt.xticks(rotation=90)  # Rotation des étiquettes de l'axe x à 90 degrés

plt.savefig(f"{results_path}{base_name}_{num_topic}t_{window_size}ws_{polynomial_order}po_sentiments_over_time.png",
            dpi=300,
            bbox_inches='tight')

plt.close()

In [ ]:
# Calcul du poids total de chaque topic par jour pour chaque topic_count
total_weight_by_topic_count_topic_and_date = {}
for topic_count, articles in every_topics_and_scores_by_document.items():
    for article_num, topic_scores in articles.items():
        article_date = dates[article_num]
        for topic_num, topic_weight in topic_scores:
            key = (topic_count, topic_num, article_date)
            if key not in total_weight_by_topic_count_topic_and_date:
                total_weight_by_topic_count_topic_and_date[key] = topic_weight
            else:
                total_weight_by_topic_count_topic_and_date[key] += topic_weight

# Initialisation d'un dictionnaire pour stocker les sentiments normalisés par date, topic_count et topic_num
sentiment_by_date_and_topic = {}

for topic_count, articles in every_topics_and_scores_by_document.items():
    for article_num, topic_scores in articles.items():
        article_date = dates[article_num]  # Date de l'article
        sentiment_score = transformed_sentiments[article_num]  # Score de sentiment de l'article

        for topic_num, topic_weight in topic_scores:
            # Ajustement du score de sentiment par le poids du topic dans l'article
            adjusted_sentiment_score = sentiment_score * topic_weight

            # Clé pour le poids total du topic par jour pour un topic_count donné
            weight_key = (topic_count, topic_num, article_date)
            if total_weight_by_topic_count_topic_and_date[weight_key] > 0:
                normalized_sentiment_score = adjusted_sentiment_score / total_weight_by_topic_count_topic_and_date[weight_key]
            else:
                normalized_sentiment_score = 0

            # Clé unique pour la combinaison topic_count, topic_num et date
            combined_key = (topic_count, topic_num, article_date)

            if combined_key not in sentiment_by_date_and_topic:
                if normalized_sentiment_score < 0:
                    sentiment_by_date_and_topic[combined_key] = [-math.sqrt(abs(normalized_sentiment_score))]
                else:
                    sentiment_by_date_and_topic[combined_key] = [math.sqrt(abs(normalized_sentiment_score))]
            else:
                if normalized_sentiment_score < 0:
                    sentiment_by_date_and_topic[combined_key].append(-math.sqrt(abs(normalized_sentiment_score)))
                else:
                    sentiment_by_date_and_topic[combined_key].append(math.sqrt(abs(normalized_sentiment_score)))

# Calcul de la moyenne des sentiments normalisés pour chaque combinaison de topic_count, topic_num et date
for key, normalized_sentiments in sentiment_by_date_and_topic.items():
    average_sentiment = sum(normalized_sentiments) # / len(normalized_sentiments)
    sentiment_by_date_and_topic[key] = average_sentiment

# **SENTIMENT OVER TIME BY TOPIC**

In [ ]:
relative_normalizaton = False
sigma = 10

In [ ]:
for topic_count, articles in every_topics_and_scores_by_document.items():
    # Filtrer les données pour un topic_count spécifique
    filtered_data = {(topic_num, date): sentiment for (count, topic_num, date), sentiment
                     in sentiment_by_date_and_topic.items() if count == topic_count}
    # Création d'un DataFrame pour les données filtrées
    df = pd.DataFrame(list(filtered_data.items()), columns=['Topic_Date', 'Sentiment'])
    df[['Topic', 'Date']] = pd.DataFrame(df['Topic_Date'].tolist(), index=df.index)
    # Pivoter le DataFrame pour la heatmap
    #print(df)
    df = df.pivot(columns="Date", index="Topic", values="Sentiment")
    #print(df)

    df.interpolate(method='linear', axis=1, inplace=True)

    list_of_series = []

    # Application du filtre gaussien
    for index, row in df.iterrows():
        filtered_values = gaussian_filter(row, sigma=sigma)

        s = pd.Series(filtered_values, index=df.columns, name=index)
        list_of_series.append(s)

    df_normalized = pd.concat(list_of_series, axis=1).T

    if relative_normalizaton:
        list_of_series = []

        for index, row in df_normalized.iterrows():
            normalized_values = (row - row.min()) / (row.max() - row.min())
            s = pd.Series(normalized_values, index=df_normalized.columns, name=index)
            list_of_series.append(s)

        df_normalized = pd.concat(list_of_series, axis=1).T

    dist_matrix = cosine_distances(df_normalized.values)
    condensed_dist_matrix = squareform(dist_matrix)

    Z = linkage(condensed_dist_matrix, method='ward', optimal_ordering=True)   # method='ward')
    dendro = dendrogram(Z, no_plot=True)
    df_normalized = df_normalized.iloc[dendro['leaves']]
    reordered_labels = [original_labels[topic_count][i] for i in np.array(df_normalized.index)]

    df_normalized.columns = pd.to_datetime(df_normalized.columns).date

    plt.figure(figsize=(16, 10))
    ax = sns.heatmap(df_normalized, cmap="coolwarm", cbar=False)
    ax.set_yticklabels(reordered_labels, rotation=0)
    plt.tight_layout()
    ax.set_xlabel('')
    ax.set_ylabel('')
    plt.savefig(f"{results_path}{base_name}_{topic_count}t_{relative_normalizaton}rn_{sigma}s_sentiments_heatmap.png",
                dpi=300,
                bbox_inches='tight')
    plt.close()

# **STACKED AREA CHARTS**

In [ ]:
def prepare_data_for_stacked_area_chart(topics_data, dates):
    # Initialiser une liste pour stocker les DataFrames temporaires
    temp_dfs = []

    # Itérer sur chaque article en vérifiant les limites du tableau dates
    for article_num, topics in topics_data.items():
        # Vérifier si l'index est dans les limites du tableau dates
        if article_num < len(dates):
            # Obtenir la date correspondante
            date = dates[article_num]

            # Créer un DataFrame temporaire pour cet article
            temp_df = pd.DataFrame([score for _, score in topics],
                                   index=[topic for topic, _ in topics],
                                   columns=[date]).T

            # Ajouter le DataFrame temporaire à la liste
            temp_dfs.append(temp_df)

    # Concaténer tous les DataFrames temporaires
    df = pd.concat(temp_dfs)

    # Regrouper par date et sommer les scores pour chaque topic
    df = df.groupby(level=0).sum()

    return df

In [ ]:
def apply_weighted_smoothing(df, column_name):
    n = len(df[column_name])
    # Define points where the weight starts to increase/decrease
    lower_bound = int(n * 0.3)
    upper_bound = int(n * 0.7)

    # Initialize weights to 1
    weights = np.ones(n)

    # Set weights: from 1 to 0 for the first 30%, remain at 0 up to 70%, then from 0 back to 1
    weights[:lower_bound] = np.linspace(1, 0, lower_bound)
    weights[lower_bound:upper_bound] = 0
    weights[upper_bound:] = np.linspace(0, 1, n - upper_bound)

    # Calculate the weighted rolling means
    smooth_data = pd.Series(df[column_name]).rolling(window=lower_bound, min_periods=1).mean()

    # Apply the custom weights to have a progressive smoothing towards the 30% mark from the ends
    # The middle values (30% to 70%) are not smoothed at all (weight of 0 on the smoothed values)
    smooth_data = smooth_data * weights + df[column_name] * (1 - weights)
    return smooth_data

In [ ]:
deg = 10

In [ ]:
for i_test in range(1):
    for topic_count in every_topics_and_scores_by_document:
        df_topics = prepare_data_for_stacked_area_chart(every_topics_and_scores_by_document[topic_count],
                                                        dates)
        df_topics.index = pd.to_datetime(df_topics.index)

        df_resampled = df_topics.resample('D').asfreq()

        # Interpolez les valeurs manquantes linéairement
        df_topics = df_resampled.interpolate(method='linear')

        # Création d'un nouveau DataFrame pour stocker les courbes ajustées
        df_smooth = pd.DataFrame(index=df_topics.index)

        x_data = np.linspace(0, 1, len(df_topics))

        for column in df_topics.columns:
            # Ajuster le polynôme aux données
            coeffs = np.polyfit(x_data, df_topics[column], deg)

            # Évaluer le polynôme ajusté
            df_smooth[column] = np.polyval(coeffs, x_data)

        dist_matrix = cosine_distances(df_smooth.transpose().values)
        cosine_dist_matrix = squareform(dist_matrix)

        # Effectuer le clustering hiérarchique à l'aide de la matrice de distances
        Z = linkage(cosine_dist_matrix, method='complete', optimal_ordering=True)

        # Obtenir l'ordre des feuilles de l'arbre de clustering
        leaf_order = leaves_list(Z)

        # Convertir leaf_order en noms de colonnes
        column_names_ordered = df_smooth.columns[leaf_order]

        # Réorganiser df_smooth en utilisant les noms des colonnes
        df_smooth = df_smooth[column_names_ordered]

        nb_try = 0
        while (df_smooth < 0).any().any() and nb_try < 500:
            nb_try += 1

            df_smooth = df_smooth.clip(lower=0)

            # Remplacer les valeurs NaN par 0 avant la normalisation si nécessaire
            df_smooth.fillna(0, inplace=True)

            if i_test == 0:
                # Calculer la somme de chaque ligne
                row_sums = df_smooth.sum(axis=1)

                # Éviter la division par zéro en remplaçant les sommes nulles par 1
                # Cela évite de diviser par zéro si une ligne entière est composée de zéros
                row_sums[row_sums == 0] = 1

                df_smooth = df_smooth.div(row_sums,
                                        axis=0)

            df_smooth2 = pd.DataFrame(index=df_topics.index)

            x_data = np.linspace(0,
                                1,
                                len(df_topics))

            for column in df_smooth.columns:
                # Ajuster le polynôme aux données
                coeffs = np.polyfit(x_data,
                                    df_smooth[column],
                                    deg)

                # Évaluer le polynôme ajusté
                df_smooth2[column] = np.polyval(coeffs,
                                                x_data)

            df_smooth = df_smooth2


        #df_smooth = df_smooth.apply(lambda col: apply_weighted_smoothing(df_smooth, col.name))


        # Tracer le graphique en aires empilées avec les données lissées
        fig, ax = plt.subplots(figsize=(20, 12))
        palette = sns.color_palette("pastel")
        stacks = ax.stackplot(df_topics.index,
                            df_smooth.T,
                            edgecolor='white',
                            linewidth=0.5,
                            colors=palette)

        colors = [stack.get_facecolor()[0] for stack in stacks]

        # Ajouter les noms des topics sur les aires
        cumulative_heights = np.cumsum(df_smooth.values,
                                    axis=1)
        x_label = df_topics.index[0]  # Position x pour les labels

        x_label_num = mdates.date2num(df_topics.index[0])

        original_labels_reordered = [original_labels[topic_count][name]
                                     for name in column_names_ordered]

        all_y = []
        all_topics = []
        for i, topic in enumerate(original_labels_reordered):
            all_topics.append(topic)
            all_y.append((cumulative_heights[0, i] +
                          cumulative_heights[0, i - 1]) / 2
                         if i > 0 else cumulative_heights[0, i] / 2)

        everything_fine = False
        while not everything_fine:
            everything_fine = True

            i = 1
            while i < len(all_y):
                correction = False
                while (all_y[i] - all_y[i - 1]) < 0.022:
                    all_y[i - 1] = all_y[i - 1] - 0.00001
                    all_y[i] = all_y[i] + 0.00001
                    correction = True

                if correction:
                    everything_fine = False

                i += 1


        offset_x = int(len(dates) / 10)
        offset_x = 0
        i = 0
        while i < len(all_y):
            ax.text(x_label_num - offset_x,
                    all_y[i],
                    all_topics[i],
                    ha='right',
                    va='center',
                    color='black',
                    fontname='Liberation Mono',
                    bbox=dict(facecolor=colors[i], pad=0.0, edgecolor='none', alpha=0.35))

            i += 1


        # Calcul de l'intervalle dynamique pour les dates
        interval = calculate_date_interval(df_topics,
                                        figsize=(20, 12))

        # Configuration de l'axe des abscisses avec l'intervalle dynamique
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=interval))

        for spine in ax.spines.values():
            spine.set_visible(False)

        ax.set_xlim(df_topics.index.min(), df_topics.index.max())

        ticks = ax.get_xticks()
        labels = [date.strftime('%d-%m-%Y') for date in mdates.num2date(ticks)]

        ax.set_xticks([])  # Supprimer les marqueurs de l'axe des y
        ax.set_yticks([])  # Supprimer les marqueurs de l'axe des y
        ax.set_xlabel('')  # Supprimer le titre de l'axe des x
        ax.set_ylabel('')  # Supprimer le titre de l'axe des y
        ax.set_title('')  # Supprimer le titre du graphique

        cumulative_max = df_smooth.sum(axis=1).max()

        # Utilisez une boucle pour positionner chaque étiquette
        for i, label in enumerate(labels):
            ax.text(ticks[i],
                    0,
               #     -0.013*cumulative_max,
                    label,
                    rotation=90,
                    ha='left',
                    va='top',
                    fontname='Liberation Mono')

        plt.ylim(top=cumulative_max)

        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        plt.tight_layout(pad=1.0, h_pad=1.0, w_pad=1.0, rect=[0, 0, 1, 1])

        if i_test == 0:
            plt.savefig(f"{results_path}{base_name}_{topic_count}t_" \
                        f"{deg}d_100_stacked_area_chart.png",
                        dpi=300,
                        bbox_inches='tight',
                        pad_inches=0)
        else:
            plt.savefig(f"{results_path}{base_name}_{topic_count}t_" \
                        f"{deg}d_stacked_area_chart.png",
                        dpi=300,
                        bbox_inches='tight',
                        pad_inches=0)

        plt.close()

# **ENTROPY DYNAMICS**

In [ ]:
def calculate_entropy(words):
    word_counts = Counter(words)
    total_words = sum(word_counts.values())
    entropy = -sum((count/total_words) * math.log2(count/total_words) for count in word_counts.values())
    return entropy

for topic_count in every_topics_and_scores_by_document:
    # Charger les données
    data = pd.read_csv(f"{results_path}{base_name}_lemmas_info_{topic_count}.csv", sep=';')

    # Convertir les dates en objets de date avec le format jour/mois/année
    data['date'] = pd.to_datetime(data['date'], dayfirst=True, errors='coerce')

    # Convertir les dates en périodes temporelles hebdomadaires
    data['week'] = data['date'].dt.to_period('W')

    # Calculer l'entropie pour chaque semaine
    entropies = data.groupby('week')['lemma'].apply(calculate_entropy)

In [ ]:
if pd.NaT in entropies.index:
    entropies = entropies.drop(pd.NaT)

# Vérifier si l'index est déjà un timestamp
if entropies.index.dtype != 'datetime64[ns]':
    entropies.index = entropies.index.to_timestamp()

In [ ]:
window_size = 2  # La taille de la fenêtre peut être ajustée selon vos besoins

In [ ]:
rolling_entropies = entropies.rolling(window=window_size, center=True, min_periods=1).mean()

# Plotting the data using seaborn
plt.figure(figsize=(20, 12))
sns.lineplot(data=rolling_entropies)

# Personnalisation
plt.title(f"entropie hebdomadaire des mots - lissage avec moyenne mobile sur une fenêtre de {window_size}")
plt.xlabel("")
plt.ylabel("entropie moyenne")
plt.xticks(rotation=90)

sns.despine()
plt.tight_layout()

# Afficher le graphique
plt.savefig(f"{results_path}{base_name}_lemmas_entropy_week_based_{window_size}ws.png")
plt.close()

# **TOPICS CLUSTERING**

In [ ]:
for topic_count in every_topics_and_scores_by_document:
    similarity_matrix = cosine_similarity(all_nmf_H[topic_count])

    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(similarity_matrix)

    # Calcul de la distance euclidienne par rapport au centre [0, 0]
    distances = np.sqrt(np.sum(pca_result ** 2, axis=1))

    # Créer une colormap pour les couleurs en fonction de la distance
    colormap = plt.cm.Blues

    # Normaliser les distances pour les utiliser comme valeurs de couleur
    norm = plt.Normalize(vmin=distances.min(), vmax=distances.max())
    colors = colormap(norm(distances))

    # Récupérer les scores de résumé de la variance (explained variance ratio)
    explained_var_ratio = pca.explained_variance_ratio_

    plt.figure(figsize=(10, 10))

    # Visualisation avec des couleurs en fonction de la distance
    plt.scatter(pca_result[:, 0], pca_result[:, 1], c=colors)

    labels = [f'{i}' for i in range(len(pca_result))]

    texts = [plt.text(topic[0], topic[1], original_labels[topic_count][int(label)]) for topic, label in zip(pca_result, labels)]

    adjust_text(texts, force_points=0.2, force_text=0.2, expand_text=(1.2, 1.2))

    # Afficher les scores de résumé de la variance sur les axes
    plt.title(f"ACP des distances cosine inter-topics\nbasées sur les vecteurs d'unigrammes")
    plt.xlabel(f'facteur 1 - variance expliquée={explained_var_ratio[0]*100:.2f}%')
    plt.ylabel(f'facteur 2 - variance expliquée={explained_var_ratio[1]*100:.2f}%')

    # Supprimer la partie haute et droite du cadre
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)

    plt.tight_layout()

    plt.savefig(f"{results_path}{base_name}_{topic_count}nc_inter_topics_cosine_pca.png")
    plt.close()